In [ ]:
import numpy as np

In [ ]:
class Loss(object):
    def forward(self, y, yhat):
        pass

    def backward(self, y, yhat):
        pass

In [ ]:
class MSELoss(Loss):
    def forward(self, y, yhat):
        return np.linalg.norm(y - yhat, axis=1)**2 # axis=1 to have batch vectors
    
    def backward(self, y, yhat):
        return -2 * (y - yhat) # taille batch*d, gradient de la MSE en fonction des yhat.

In [ ]:
class CE(Loss):
    def forward(self, y, yhat): # y and yhat are matrix
        onehot = np.zeros((y.shape[1],10))
        onehot[np.arange(y.size),y]=1
        
        return np.array([-yhat[i][onehot[i]==1] for i in range(y.shape[0])])
    
    def backward(self, y, yhat):
        return 

In [ ]:
class Module(object):
    def __init__(self):
        self._parameters = None
        self._gradient = None

    def zero_grad(self):
        ## Annule gradient
        pass

    def forward(self, X):
        ## Calcule la passe forward
        pass

    def update_parameters(self, gradient_step=1e-3):
        ## Calcule la mise a jour des parametres selon le gradient calcule et le pas de gradient_step
        self._parameters -= gradient_step*self._gradient

    def backward_update_gradient(self, input, delta):
        ## Met a jour la valeur du gradient
        pass

    def backward_delta(self, input, delta):
        ## Calcul la derivee de l'erreur
        pass

In [ ]:
class Linear(Module):
    def __init__(self, input, output):
        super().__init__(self)
        self.input = input
        self.output = output
        self._parameters = np.zeros((input, output)) # (batch*input @ input*output) -> (batch*output)
        self._gradient = np.zeros(self._parameters.shape)
    
    def zero_grad(self): # reinitialise à 0 le gradient
        return np.zeros(self._parameters.shape)
    
    def forward(self, X):
        return X @ self._parameters # sans biais
        
    def backward_update_gradient(self, X, delta):
        self._gradient += X.T @ delta
        
    def backward_delta(self, X, delta):
        return delta @ self._parameters.T

In [ ]:
class TanH(Module):
    def __init__(self):
        super().__init__(self)
        
    def forward(self, X):
        return np.tanh(X) # transformation
    
    def backward_update_gradient(self, X, delta):
        return 1 - self.forward(X)**2 * delta # d'après la dérivée en fonction de X de tanh(X), delta joue le rôle de constante.
    
    def update_parameters(self, gradient_step=1e-3):
        pass # nous n'avons pas de paramètre dans ce modèle

In [ ]:
def sigmoide(X):
    return 1/(1+np.exp(-X)) # sigmoide n'existe pas dans NumPy, nous l'implémentons ici.

class Sigmoide(Module):
    def __init__(self):
        super().__init__(self)
        
    def forward(self, X):
        return sigmoide(X)
    
    def backward_update_gradient(self, X, delta):
        return (np.exp(-X) * sigmoide(X) / (1+np.exp(-X))) * delta # d'après la dérivée en fonction de X de sigmoide(X) (avec une constante lambda de 1), delta joue le rôle de constante.
    
    def update_parameters(self, gradient_step=1e-3):
        pass # nous n'avons pas de paramètre dans ce modèle

In [ ]:
class Softmax(Module):
    def __init__(self):
        super().__init__(self)
        
    def forward(self, Z): # Z is a vector
        e = np.exp(Z)
        return e / np.sum(e)
    
    def backward_update_gradient(self, Z, delta): # Jacobian of Softmax function
        K = Z.size
        jcb_softmax = np.zeros((K,K))
        softmax = self.forward(Z)
        for i in range(K):
            for j in range(K):
                if i==j:
                    jcb_softmax[i][j] = softmax[i] * (1 - softmax[i])
                else:
                    jcb_softmax[i][j] = - softmax[i] * softmax[j]
                    
        return jcb_softmax
    
    def update_parameters(self, gradient_step=1e-3):
        pass # nous n'avons pas de paramètre dans ce modèle

In [ ]:
class Sequentiel(object):
    